In [2]:
import pandas as pd
import numpy as np


# Kanton

In [ ]:

kanton = './data/plz_verzeichnis.csv'
df_kanton = pd.read_csv(kanton, sep=';')
#df_kanton.columns = df_cntrycd.columns.str.lower()
display(df_kanton.shape)
display(df_kanton.head())

# SUBSCRIPTIONS

In [3]:
# define the name of the csv file you want to read in
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'

In [4]:
# Read subscriptions
df_subscriptions = pd.read_csv(subscriptions)
df_subscriptions.columns = df_subscriptions.columns.str.lower()

display(df_subscriptions.shape)
display(df_subscriptions.head(10))

(12115, 9)

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription
0,1b82b2308dcae546ad0194d03b9d23edd5d2781109de28...,ab1bcf2c1b8d4eb422079becc0c59b1c7db2fd009235d2...,chf,90.0,BASIC,0,2017-02-24,2021-02-23,False
1,4cb9f67d44d60eca0485270d73b3028143021296081847...,ee1a45a439dc6968f1cfc2b9840264deeada4c0d93f389...,chf,90.0,BASIC,0,2017-08-21,2023-08-20,False
2,40f9e218b70a6f482ecfcde0011305d93fa6059f624a50...,8bf8f58e615549df5f05b717b6e914da80e213edab3dba...,chf,9.0,BASIC,1,2017-09-26,2022-10-08,False
3,69d21e601612cc97fc7df9bd17a281879f785358d1c681...,cf914aa939499e8e18fdd9494fce5297b9dce2e1f909ea...,chf,90.0,BASIC,0,2017-09-28,2021-09-27,False
4,29f98a3154a679f2fe953b1f3b148ddffcb28e60fff719...,8759ab0e44c1cf688770220cf1b87efac2726baca6bfcb...,eur,11.0,STANDARD,1,2017-10-13,2021-10-12,False
5,f5627e4ef66f219cfbcfb507a056bf1a0629f085a46e92...,c5511d59d31159ecb87ca1d7cfb3e44a6c4a4b96e5632a...,chf,240.0,PATRON,0,2017-11-18,2020-11-17,False
6,8e42bc7151b4b0dd29e362b95fec4aa3b8e53c532d40ba...,7f1182ed60acb48e4a55eb54b91bca7f20c55c066d8848...,chf,150.0,STANDARD,0,2017-12-20,2021-12-19,False
7,9f22270608209a507f93f3a74448a78a82f9741bc36ea3...,2c2621bf34f1cc419c0868f27ffece66dc1fb4f5ec665c...,chf,9.0,BASIC,1,2017-12-21,2020-11-08,False
8,03411f3ad7d86f2e1df3db5a337c3025ba35159eb9b41e...,163db260041d30a58540a19312d416c9fc7cc696754108...,chf,9.0,BASIC,1,2017-12-22,2020-10-08,False
9,c254cee83dfc67c794b843a0ad622456cb36fdfe5143c4...,47e795383bff3d37a251be1d911ffc0a7ca6f0da8b70a4...,chf,240.0,PATRON,0,2017-12-31,2022-12-30,False


In [5]:
df_subscriptions.groupby(['currency','price']).subscription_type.value_counts()

currency  price  subscription_type
chf       0.0    PATRON                  7
                 BASIC                   6
                 STANDARD                5
          9.0    BASIC                2917
          15.0   STANDARD             2118
          27.0   BASIC                   1
          67.5   STANDARD                1
          75.0   STANDARD                2
          82.0   BASIC                   1
          90.0   BASIC                2533
                 FULLACCESS              1
                 STANDARD                1
          102.5  PATRON                  2
          150.0  STANDARD             1146
          172.5  PATRON                  1
          192.0  PATRON                  1
          195.0  PATRON                  1
          210.0  PATRON                  1
          220.0  PATRON                  1
          240.0  PATRON               1052
eur       6.0    BASIC                   1
          7.5    BASIC                 272
          11.0   ST

In [10]:
#set data types for subscription dates to datetime
df_subscriptions['subscription_start'] = pd.to_datetime(df_subscriptions['subscription_start'])
df_subscriptions['subscription_end'] = pd.to_datetime(df_subscriptions['subscription_end'])

In [12]:
# Create new 'clean' DataFrame 
df_subscriptions_clean = df_subscriptions

In [13]:
# drop "wrong" subscription type line (FULLACCESS - unknown type to us - only one line therefore decided to drop)
df_subscriptions_clean.drop(df_subscriptions_clean[(df_subscriptions['subscription_type'] == 'FULLACCESS')].index, inplace = True)

In [14]:
#create new column and calculate subscription duration for calculating actual price per subscription
df_subscriptions_clean['subscription_months_raw'] = ((df_subscriptions_clean.subscription_end) - df_subscriptions_clean.subscription_start)/np.timedelta64(1, 'M')

In [15]:
#create new column with rounded subscription months
#generally round up from 0.1 to be able to allow some discrepancies due to day to day calculation of subscription duration
df_subscriptions_clean['subscription_months'] = df_subscriptions_clean['subscription_months_raw'] - 0.1
df_subscriptions_clean['subscription_months'] = df_subscriptions_clean['subscription_months'].apply(np.ceil)

In [ ]:
# Create two columns for chf and eur based on the subscription_type and prices from the filmingo website

# create a list of our conditions
conditions = [
    ((df_subscriptions_clean['subscription_type'] == 'BASIC') & (df_subscriptions_clean['subscription_monthly'] == 0)),
    ((df_subscriptions_clean['subscription_type'] == 'BASIC') & (df_subscriptions_clean['subscription_monthly'] == 1)),
    ((df_subscriptions_clean['subscription_type'] == 'STANDARD') & (df_subscriptions_clean['subscription_monthly'] == 0)),
    ((df_subscriptions_clean['subscription_type'] == 'STANDARD') & (df_subscriptions_clean['subscription_monthly'] == 1)),
    ((df_subscriptions_clean['subscription_type'] == 'PATRON') & (df_subscriptions_clean['subscription_monthly'] == 0))

]

# create a list of the values we want to assign for each condition
values_chf = ['90.0', '9.0', '150.0', '15.0', '240.0']
values_eur = ['75.0', '7.5', '125.0', '12.5', '200.0']

# create a new column and use np.select to assign values to it using our lists as arguments
df_subscriptions_clean['price_chf'] = np.select(conditions, values_chf)
df_subscriptions_clean['price_eur'] = np.select(conditions, values_eur)

# decided to use these prices for all subscriptions regardless if they might have a different prices in the list (possibly due to discounts, total lines of abnormal prices: 39) or are gifted subscription (price: NaN, total lines 1.636)


In [ ]:
# if gifted & 6 monate then 41€ else price /12 * sub_months
def price_calc (x):
    if gift_subscription =

In [16]:
df_subscriptions_clean.head()

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription,subscription_months_raw,subscription_months
0,1b82b2308dcae546ad0194d03b9d23edd5d2781109de28...,ab1bcf2c1b8d4eb422079becc0c59b1c7db2fd009235d2...,chf,90.0,BASIC,0,2017-02-24,2021-02-23,False,47.968131,48.0
1,4cb9f67d44d60eca0485270d73b3028143021296081847...,ee1a45a439dc6968f1cfc2b9840264deeada4c0d93f389...,chf,90.0,BASIC,0,2017-08-21,2023-08-20,False,71.952196,72.0
2,40f9e218b70a6f482ecfcde0011305d93fa6059f624a50...,8bf8f58e615549df5f05b717b6e914da80e213edab3dba...,chf,9.0,BASIC,1,2017-09-26,2022-10-08,False,60.387277,61.0
3,69d21e601612cc97fc7df9bd17a281879f785358d1c681...,cf914aa939499e8e18fdd9494fce5297b9dce2e1f909ea...,chf,90.0,BASIC,0,2017-09-28,2021-09-27,False,47.968131,48.0
4,29f98a3154a679f2fe953b1f3b148ddffcb28e60fff719...,8759ab0e44c1cf688770220cf1b87efac2726baca6bfcb...,eur,11.0,STANDARD,1,2017-10-13,2021-10-12,False,47.968131,48.0


In [ ]:
#display(gifted_df.groupby('subscription_months').max())
print(df_subscriptions_clean.groupby(['months','subscription_monthly']).sum())

In [ ]:
df_subscriptions_clean.to_csv('./data/subcriptions_clean.csv')

In [ ]:
gifted_df= df_subscriptions_clean.loc[df_subscriptions_clean['gift_subscription']==True]
gifted_df['months'] = gifted_df['subscription_months'].round()
gifted_df.tail(15)
#display(gifted_df.groupby('subscription_months').max())
gifted_df.groupby('months').sum()



In [ ]:
nongifted_df= df_subscriptions_clean.loc[df_subscriptions_clean['gift_subscription']==False]

nongifted_df.groupby(['months', 'subscription_monthly']).sum()

In [ ]:
#nongifted_df.query('account_key == b43a87d35bf285afdbb1c931b68ea2e6dad1f9dcc62947')

nongifted_df[nongifted_df['account_key'].str.contains('d8aa9f6793e94bc168a65808c9fe5809d4516448eae392a33edec16391c71d1e', na=False)]

#due to update rule ab 0.1 aufrunden

In [ ]:
b43a87d35bf285afdbb1c931b68ea2e6dad1f9dcc62947...	Account Key	Gift Subscription	Months	Price Eur	Subscription Monthly	Subscription Months	Subscription Type
d8aa9f6793e94bc168a65808c9fe5809d4516448eae392a33edec16391c71d1e	False	36	75	0	36,206082260	BASIC


In [ ]:

nongifted_df['months_rounded'] = nongifted_df['subscription_months'] - 0.1
nongifted_df['months_ceil'] = nongifted_df['months_rounded'].apply(np.ceil)

In [ ]:
#df_subscriptions_clean = df_subscriptions_clean.drop('months_rounded_ceil', axis=1)